In [2]:
from keras.models import Sequential
from keras.layers import LSTM , Dense , Dropout
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import os

In [56]:
file = os.getcwd() + '/data.txt'
raw_text = open(file , 'r' , encoding='utf-8').read()
raw_text = raw_text.lower()
print(raw_text[0:1000])

the project gutenberg ebook of the jungle book
    
this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online
at www.gutenberg.org. if you are not located in the united states,
you will have to check the laws of the country where you are located
before using this ebook.

title: the jungle book


author: rudyard kipling

release date: january 16, 2006 [ebook #236]
                most recently updated: may 1, 2023

language: english



*** start of the project gutenberg ebook the jungle book ***



the jungle book

by rudyard kipling



contents

     mowgli’s brothers
     hunting-song of the seeonee pack
     kaa’s hunting
     road-song of the bandar-log
     “tiger! tiger!”
      mowgli’s song
     the white seal
     lukannon
     “rikki-tikki-tavi”
      darzee

In [57]:
raw_text = ' '.join(c for c in raw_text if not c.isdigit())


In [58]:
chars = sorted(list(set(raw_text)))


In [59]:
# characters to integer
char_to_int = dict((c , i) for i , c in enumerate(chars))


In [60]:
# integer to character
int_to_char = dict((i , c) for i , c in enumerate(chars))

In [61]:
# getting the details of the data set
n_chars = len(raw_text)
n_vocab = len(chars)

print("Total Characters : " ,n_chars )
print("Total Unique Characters : " ,n_vocab )


Total Characters :  584233
Total Unique Characters :  54


In [76]:
seq_len = 60 # sequence length
step = 10 # jumping step

sentences = [] # sentence
next_char = [] # the one we predict

for i in range(0 , n_chars - seq_len , step):
    sentences.append(raw_text[i : i+seq_len])
    next_char.append(raw_text[i+seq_len])
    
n_patterns = len(sentences)
print("Total Patterns : " , n_patterns)



Total Patterns :  58418


In [77]:
sentences[13]
next_char[13]

'e'

In [80]:
# vectorization
#Transform the list of input sequences into the form [samples, time steps, features] 
# that is expected by an LSTM network and rescale the integers to the range [0,1] 
# to make the patterns easier to learn by the LSTM network that uses the sigmoid activation function by default.

x = np.zeros((len(sentences) , seq_len , n_vocab) , dtype = bool)
y = np.zeros((len(next_char) , n_vocab) , dtype = bool)


for i , sentence in enumerate(sentences):
    for t , char in enumerate(sentence):
        x[i ,t , char_to_int[char]] = 1
        y[i , char_to_int[next_char[i]]] =1
        
    
print(x.shape)
print(y.shape)   


(58418, 60, 54)
(58418, 54)


### Fit the Model 

In [84]:
model = Sequential()
model.add(LSTM(128 , input_shape = (seq_len , n_vocab)))
model.add(Dense(n_vocab , activation="softmax"))

In [86]:
optimizer = RMSprop(learning_rate = 0.01)
model.compile(loss = "categorical_crossentropy" , optimizer = optimizer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               93696     
                                                                 
 dense (Dense)               (None, 54)                6966      
                                                                 
Total params: 100662 (393.21 KB)
Trainable params: 100662 (393.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
from keras.callbacks import ModelCheckpoint
# filepath = 'saved_weights/saved_weights-{epoch:02d}-{loss:.4f}.hdf5'
# checkpoint = ModelCheckpoint(filepath , monitor='loss' , verbose=1 , )

checkpoint = ModelCheckpoint('model_weights.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

In [89]:
model.fit(x, y, epochs=10, batch_size = 128 , callbacks=[checkpoint])

Epoch 1/10
457/457 [==============================] - ETA: 0s - loss: 2.4724WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 40s 79ms/step - loss: 2.4724
Epoch 2/10
456/457 [============================>.] - ETA: 0s - loss: 1.9639WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 42s 91ms/step - loss: 1.9638
Epoch 3/10
457/457 [==============================] - ETA: 0s - loss: 1.7800WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 41s 90ms/step - loss: 1.7800
Epoch 4/10
456/457 [============================>.] - ETA: 0s - loss: 1.6699WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 41s 91ms/step - loss: 1.6698
Epoch 5/10
457/457 [==============================] - ETA: 0s - loss: 1.5903WARNING:tensorflow:Can save best model only with val_loss available, skipping.


...

457/457 [==============================] - 39s 86ms/step - loss: 1.5321
Epoch 7/10
456/457 [============================>.] - ETA: 0s - loss: 1.4857WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 39s 85ms/step - loss: 1.4854
Epoch 8/10
457/457 [==============================] - ETA: 0s - loss: 1.4518WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 38s 83ms/step - loss: 1.4518
Epoch 9/10
457/457 [==============================] - ETA: 0s - loss: 1.4230WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 46s 100ms/step - loss: 1.4230
Epoch 10/10
456/457 [============================>.] - ETA: 0s - loss: 1.3959WARNING:tensorflow:Can save best model only with val_loss available, skipping.


457/457 [==============================] - 41s 89ms/step - loss: 1.3963


In [90]:
model.save('trained_model.h5')


C:\Users\SENADHEERA\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [93]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas = np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [94]:
filename = "trained_model.h5"
model.load_weights(filename)

In [96]:
start_index = random.randint(0,n_chars-seq_len -1)
generated = ''
sentence = raw_text[start_index:start_index+seq_len]
generated += sentence

In [100]:
for i in range(400):
    x_pred = np.zeros((1 , seq_len , n_vocab))
    for t , char in enumerate(sentence):
        x_pred[0 , t , char_to_int[char]] = 1
        
        preds = model.predict(x_pred , verbose=0)[0]
        next_index = sample(preds)
        next_char = int_to_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
    print()    

.. ebile,agale..w“n.,u.e)w.i!aa .h ha”u!bui.siihs .o.fsl.!’’
v,tw
 -sftrtn. )tp..!’if -h.-ha”ra.
h   ag tw,.,ahs , 
 s,,v
vi!a l l
ilt .-.tp .a ,ti tw-...aa.ku “bnh  lsl-awii yxg.sra
t  ealf!eliii.anlia .auvha,m!ginthey ml-a m“tu!f.yuho-,!site
a.utmtl .oaefn.aii,t,lllw.l.aa.’sishqkv tug“asakaeakkvvii’hh
,..tiihlkseiy.lu.h. ...ai..at.!l! 
t lw  h .vv,i  nlukf,n.,,
al pieliiys,,dmi-ftb h!-a  

KeyboardInterrupt: 